<h1>Clustering articles using Topic Modelling</h1>

In [97]:
#Loading required packages
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 200)

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [98]:
import gensim
from gensim.models import CoherenceModel
import pyLDAvis.gensim

import spacy
nlp = spacy.load('en')

In [99]:
train = pd.read_pickle("corpus.pkl")
train.head()

,title,original_content,content,tokenized_content,clean_list_tokenized_content,clean_string_tokenized_content
number,,,,,,
Article 1,Pier 1 appoints interim CFO amid growing financial loss,"\n\nPier 1 on Wednesday reported that fourth quarter net sales fell 19.5% to $412.5 million from $512.2 million in the year-ago quarter. Comp store sales in the quarter decreased 13.7%, which the ...",Pier on Wednesday reported that fourth quarter net sales fell to million from million in the yearago quarter Comp store sales in the quarter decreased which the company attributes partially to the...,"(Pier, on, Wednesday, reported, that, fourth, quarter, net, sales, fell, to, million, from, million, in, the, yearago, quarter, Comp, store, sales, in, the, quarter, decreased, which, the, company...","[pier, Wednesday, report, fourth, quarter, net, sale, fall, million, million, yearago, quarter, Comp, sale, quarter, decrease, attribute, partially, shift, holiday, selling, day, include, fourth, ...","pier, Wednesday, report, fourth, quarter, net, sale, fall, million, million, yearago, quarter, Comp, sale, quarter, decrease, attribute, partially, shift, holiday, selling, day, include, fourth, q..."
Article 2,Family Dollar to close nearly 400 stores,\nDollar Tree on Wednesday announced that up to 390 Family Dollar stores will close in 2019. The company closed 84 Family Dollar stores in the fourth quarter.\nThe retailer also reported consolida...,Dollar Tree on Wednesday announced that up to Family Dollar stores will close in The company closed Family Dollar stores in the fourth quarter The retailer also reported consolidated net sales for...,"(Dollar, Tree, on, Wednesday, announced, that, up, to, Family, Dollar, stores, will, close, in, The, company, closed, Family, Dollar, stores, in, the, fourth, quarter, The, retailer, also, reporte...","[dollar, Tree, Wednesday, announce, Family, Dollar, close, close, Family, Dollar, fourth, quarter, report, consolidated, net, sale, week, fourth, quarter, fall, billion, billion, previous, year, w...","dollar, Tree, Wednesday, announce, Family, Dollar, close, close, Family, Dollar, fourth, quarter, report, consolidated, net, sale, week, fourth, quarter, fall, billion, billion, previous, year, we..."
Article 3,Having to share personal data turns consumers away from loyalty programs study says,"\nAccording to a new Harris Poll survey, 71% of Americans would be less likely to join a rewards program that collects personal information, such as address, account information and more, with 27%...",According to a new Harris Poll survey of Americans would be less likely to join a rewards program that collects personal information such as address account information and more with saying they w...,"(According, to, a, new, Harris, Poll, survey, of, Americans, would, be, less, likely, to, join, a, rewards, program, that, collects, personal, information, such, as, address, account, information,...","[new, Harris, Poll, survey, Americans, likely, join, reward, program, collect, personal, information, address, account, information, say, likely, join, additionally, reveal, likely, join, program,...","new, Harris, Poll, survey, Americans, likely, join, reward, program, collect, personal, information, address, account, information, say, likely, join, additionally, reveal, likely, join, program, ..."
Article 4,Walgreens taps Narvar for online pickup returns in 8K stores,\n\np.p1 {margin: 0.0px 0.0px 35.0px 0.0px; line-height: 28.0px; font: 16.0px Helvetica; color: #2a2a2a; -webkit-text-stroke: #2a2a2a; background-color: #ffffff}\r\nspan.s1 {font-kerning: none}\r\...,Customer experience platform Narvar and Walgreens are working together to provide package pickup and returns at more than Walgreens locations that have FedEx OnSite services Its the first offering...,"(Customer, experience, platform, Narvar, and, Walgreens, are, working, together, to, provide, package, pickup, and, returns, at, more, than, Walgreens, locations, that, hav

<h3>1. Running LDA - Using all Text</h3>

In [100]:
#Create a dictionary from containing the number of times a word appears 
dictionary = gensim.corpora.Dictionary(train['clean_list_tokenized_content'])

In [101]:
'''
Create the Bag-of-words model for each article i.e for each article we create a dictionary reporting how many
words and how many times those words appear.
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in train['clean_list_tokenized_content']]

In [102]:
model1 = gensim.models.LdaModel(bow_corpus, id2word=dictionary, num_topics=3, passes=10)
model1.print_topics()

[(0,
  '0.018*"million" + 0.014*"quarter" + 0.011*"brand" + 0.010*"year" + 0.009*"sale" + 0.008*"net" + 0.008*"include" + 0.006*"say" + 0.006*"program" + 0.006*"yearago"'),
 (1,
  '0.011*"report" + 0.010*"year" + 0.008*"sale" + 0.006*"week" + 0.006*"say" + 0.006*"president" + 0.005*"announce" + 0.005*"executive" + 0.005*"quarter" + 0.005*"Thursday"'),
 (2,
  '0.013*"say" + 0.010*"year" + 0.009*"revenue" + 0.007*"technology" + 0.006*"new" + 0.006*"chain" + 0.006*"supply" + 0.006*"million" + 0.005*"report" + 0.005*"increase"')]

In [103]:
model2 = gensim.models.LdaModel(bow_corpus, id2word=dictionary, num_topics=4, passes=10)
model2.print_topics()

[(0,
  '0.029*"million" + 0.026*"quarter" + 0.022*"year" + 0.021*"sale" + 0.018*"net" + 0.011*"rise" + 0.011*"billion" + 0.009*"report" + 0.009*"comp" + 0.008*"fourth"'),
 (1,
  '0.010*"include" + 0.008*"brand" + 0.008*"new" + 0.007*"program" + 0.006*"report" + 0.006*"say" + 0.005*"supply" + 0.005*"chain" + 0.005*"million" + 0.005*"announce"'),
 (2,
  '0.012*"say" + 0.009*"survey" + 0.008*"join" + 0.008*"find" + 0.007*"new" + 0.007*"brand" + 0.006*"inventory" + 0.006*"respondent" + 0.005*"video" + 0.005*"customer"'),
 (3,
  '0.010*"report" + 0.010*"say" + 0.009*"Walmart" + 0.007*"year" + 0.006*"brand" + 0.006*"technology" + 0.006*"’s" + 0.005*"time" + 0.005*"president" + 0.004*"include"')]

<h3>2. Running LDA - Using Nouns and Adjectives only</h3>

In [104]:
tokens = []

for doc in nlp.pipe(train['clean_list_tokenized_content'].astype('unicode').values, batch_size=50, n_threads=3):
    if doc.is_parsed:
        tokens.append([n.lemma_ for n in doc if n.is_stop == False and n.pos_ in ('NOUN', 'ADJ')])      
    else:
        tokens.append(None)

train['content_NA'] = tokens  

In [105]:
dictionary_NA = gensim.corpora.Dictionary(train['content_NA'])

bow_corpus_NA = [dictionary_NA.doc2bow(doc) for doc in train['content_NA']]

In [106]:
model3 = gensim.models.LdaModel(bow_corpus_NA, id2word=dictionary_NA, num_topics=3, passes=10, random_state=42)
model3.print_topics()

[(0,
  '0.016*"brand" + 0.015*"report" + 0.011*"website" + 0.009*"revenue" + 0.009*"year" + 0.007*"digital" + 0.007*"software" + 0.007*"product" + 0.006*"strategy" + 0.006*"ecommerce"'),
 (1,
  '0.047*"\'" + 0.031*"quarter" + 0.028*"year" + 0.025*"sale" + 0.021*"net" + 0.012*"report" + 0.011*"comp" + 0.010*"fourth" + 0.010*"loss" + 0.010*"week"'),
 (2,
  '0.012*"supply" + 0.012*"technology" + 0.011*"new" + 0.011*"inventory" + 0.010*"chain" + 0.009*"brand" + 0.008*"respondent" + 0.007*"year" + 0.007*"customer" + 0.007*"report"')]

In [107]:
model4 = gensim.models.LdaModel(bow_corpus_NA, id2word=dictionary_NA, num_topics=4, passes=10, random_state=42)
model4.print_topics()

[(0,
  '0.018*"brand" + 0.012*"report" + 0.011*"website" + 0.011*"revenue" + 0.008*"digital" + 0.008*"year" + 0.008*"product" + 0.007*"strategy" + 0.007*"software" + 0.007*"member"'),
 (1,
  '0.049*"\'" + 0.034*"quarter" + 0.030*"year" + 0.027*"sale" + 0.023*"net" + 0.012*"comp" + 0.012*"report" + 0.011*"fourth" + 0.011*"brand" + 0.010*"week"'),
 (2,
  '0.016*"supply" + 0.014*"inventory" + 0.013*"chain" + 0.013*"new" + 0.011*"technology" + 0.010*"report" + 0.009*"chief" + 0.009*"ceo" + 0.008*"executive" + 0.008*"respondent"'),
 (3,
  '0.010*"report" + 0.009*"fall" + 0.009*"\'" + 0.008*"brand" + 0.008*"shopping" + 0.008*"digitalization" + 0.008*"boost" + 0.007*"year" + 0.007*"technology" + 0.007*"service"')]

<h3>3. Running LDA - Using only Nouns</h3>

In [108]:
tokens = []

for doc in nlp.pipe(train['clean_list_tokenized_content'].astype('unicode').values, batch_size=50, n_threads=3):
    if doc.is_parsed:
        tokens.append([n.lemma_ for n in doc if n.is_stop == False and n.pos_ == "NOUN"])      
    else:
        tokens.append(None)

train['content_NN'] = tokens                   

In [109]:
dictionary_NN = gensim.corpora.Dictionary(train['content_NN'])

bow_corpus_NN = [dictionary_NN.doc2bow(doc) for doc in train['content_NN']]

In [110]:
model5 = gensim.models.LdaModel(bow_corpus_NN, id2word=dictionary_NN, num_topics=3, passes=50, random_state=42)
model5.print_topics()

[(0,
  '0.052*"\'" + 0.034*"quarter" + 0.034*"year" + 0.027*"sale" + 0.023*"net" + 0.022*"report" + 0.013*"fall" + 0.013*"week" + 0.012*"comp" + 0.012*"increase"'),
 (1,
  '0.014*"brand" + 0.014*"inventory" + 0.013*"president" + 0.011*"ceo" + 0.009*"executive" + 0.008*"customer" + 0.008*"year" + 0.008*"vice" + 0.008*"report" + 0.007*"technology"'),
 (2,
  '0.018*"supply" + 0.016*"reward" + 0.016*"program" + 0.016*"loyalty" + 0.015*"chain" + 0.013*"brand" + 0.013*"card" + 0.011*"technology" + 0.010*"respondent" + 0.010*"food"')]

In [111]:
print('\nPerplexity:',model5.log_perplexity(bow_corpus_NN))

coherence_model_lda = CoherenceModel(model=model5, texts=train['content_NN'], dictionary=dictionary_NN, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity: -6.331609790412156

Coherence Score:  0.4433278535196188


In [112]:
model6 = gensim.models.LdaModel(bow_corpus_NN, id2word=dictionary_NN, num_topics=4, passes=50, random_state=42)
model6.print_topics()

[(0,
  '0.056*"\'" + 0.037*"quarter" + 0.036*"year" + 0.030*"sale" + 0.025*"net" + 0.024*"report" + 0.014*"fall" + 0.014*"week" + 0.013*"comp" + 0.013*"increase"'),
 (1,
  '0.019*"brand" + 0.013*"customer" + 0.012*"year" + 0.012*"president" + 0.010*"inventory" + 0.009*"technology" + 0.009*"color" + 0.009*"\'" + 0.009*"ceo" + 0.008*"lead"'),
 (2,
  '0.016*"card" + 0.014*"food" + 0.011*"supply" + 0.011*"technology" + 0.009*"reward" + 0.009*"program" + 0.009*"loyalty" + 0.009*"member" + 0.009*"email" + 0.009*"point"'),
 (3,
  '0.019*"survey" + 0.017*"supply" + 0.017*"respondent" + 0.014*"chain" + 0.013*"brand" + 0.013*"inventory" + 0.011*"executive" + 0.011*"president" + 0.011*"conduct" + 0.010*"board"')]

In [113]:
print('\nPerplexity:',model6.log_perplexity(bow_corpus_NN))

coherence_model_lda = CoherenceModel(model=model6, texts=train['content_NN'], dictionary=dictionary_NN, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity: -6.345103350341143

Coherence Score:  0.4356523469330349


In [114]:
model7 = gensim.models.LdaModel(bow_corpus_NN, id2word=dictionary_NN, num_topics=5, passes=50, random_state=42)
model7.print_topics()

[(0,
  '0.025*"report" + 0.013*"brand" + 0.012*"video" + 0.010*"year" + 0.010*"share" + 0.009*"day" + 0.008*"website" + 0.008*"poll" + 0.008*"view" + 0.008*"sticker"'),
 (1,
  '0.022*"brand" + 0.014*"customer" + 0.013*"inventory" + 0.011*"product" + 0.011*"color" + 0.010*"technology" + 0.010*"location" + 0.010*"experience" + 0.010*"member" + 0.009*"program"'),
 (2,
  '0.017*"card" + 0.017*"technology" + 0.015*"food" + 0.014*"supply" + 0.012*"industry" + 0.011*"chain" + 0.011*"launch" + 0.010*"website" + 0.010*"program" + 0.010*"point"'),
 (3,
  '0.024*"survey" + 0.022*"respondent" + 0.019*"supply" + 0.016*"brand" + 0.015*"chain" + 0.014*"inventory" + 0.012*"conduct" + 0.010*"revenue" + 0.010*"program" + 0.010*"reward"'),
 (4,
  '0.073*"\'" + 0.045*"quarter" + 0.042*"year" + 0.033*"sale" + 0.030*"net" + 0.021*"report" + 0.017*"fall" + 0.016*"comp" + 0.015*"loss" + 0.015*"increase"')]

In [115]:
print('\nPerplexity:',model7.log_perplexity(bow_corpus_NN))

coherence_model_lda = CoherenceModel(model=model7, texts=train['content_NN'], dictionary=dictionary_NN, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity: -6.319697048864377

Coherence Score:  0.41779635351286776


<b> Model-7 which uses only Nouns and having 5 topics gives better representation of articles.</b> 

We will use Model-7 to determine the topic for each article

In [116]:
train_prediction = model7[bow_corpus_NN]
list([a for a in train_prediction])

[[(4, 0.9881705)],
 [(4, 0.9872629)],
 [(3, 0.9753664)],
 [(1, 0.98209023)],
 [(1, 0.85369915), (4, 0.12671894)],
 [(3, 0.9858564)],
 [(1, 0.98349607)],
 [(2, 0.925033), (4, 0.060903598)],
 [(4, 0.9859068)],
 [(4, 0.98776895)],
 [(4, 0.9709212)],
 [(1, 0.9835205)],
 [(0, 0.010355058),
  (1, 0.010196938),
  (2, 0.010056011),
  (3, 0.010369176),
  (4, 0.9590228)],
 [(4, 0.9803129)],
 [(4, 0.9835853)],
 [(2, 0.9722197)],
 [(4, 0.96998256)],
 [(2, 0.9820835)],
 [(4, 0.9845772)],
 [(2, 0.9802463)],
 [(1, 0.9855438)],
 [(1, 0.97879106)],
 [(4, 0.97555137)],
 [(2, 0.98630095)],
 [(4, 0.9860476)],
 [(0, 0.98648316)],
 [(3, 0.9886195)],
 [(0, 0.97537404)],
 [(0, 0.98580605)],
 [(3, 0.97817683)],
 [(1, 0.9678552)],
 [(4, 0.9801128)],
 [(1, 0.66518956), (4, 0.31588838)],
 [(3, 0.98278314)],
 [(1, 0.973666)],
 [(4, 0.9708403)],
 [(0, 0.9873854)],
 [(0, 0.9761789)],
 [(1, 0.5834946), (3, 0.39697617)],
 [(1, 0.98254484)],
 [(4, 0.98016965)],
 [(0, 0.9768838)],
 [(4, 0.98025376)],
 [(4, 0.9805665)],


In [117]:
#Identify the dominant topic in each article 
def format_topics_sentences(ldamodel, corpus):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic and Perc Contribution for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  
                wp = ldamodel.show_topic(topic_num)
                sent_topics_df = sent_topics_df.append(pd.Series([topic_num, round(prop_topic,2)]), ignore_index=True)
            else:
                break
    
    sent_topics_df.columns = ['dominant_topic', 'contribution %']
    return(sent_topics_df)

train.reset_index(inplace = True)
train_topic_model = pd.concat([train['number'], train['title'], train['original_content'],train['content'], 
                               format_topics_sentences(model7, bow_corpus_NN)], axis = 1)
train_topic_model['dominant_topic'] = train_topic_model['dominant_topic'].astype('int64')
train_topic_model.set_index('number', inplace=True)
train_topic_model = train_topic_model[['title', 'original_content', 'content', 'dominant_topic', 'contribution %']]

train_topic_model.head()

,title,original_content,content,dominant_topic,contribution %
number,,,,,
Article 1,Pier 1 appoints interim CFO amid growing financial loss,"\n\nPier 1 on Wednesday reported that fourth quarter net sales fell 19.5% to $412.5 million from $512.2 million in the year-ago quarter. Comp store sales in the quarter decreased 13.7%, which the ...",Pier on Wednesday reported that fourth quarter net sales fell to million from million in the yearago quarter Comp store sales in the quarter decreased which the company attributes partially to the...,4,0.99
Article 2,Family Dollar to close nearly 400 stores,\nDollar Tree on Wednesday announced that up to 390 Family Dollar stores will close in 2019. The company closed 84 Family Dollar stores in the fourth quarter.\nThe retailer also reported consolida...,Dollar Tree on Wednesday announced that up to Family Dollar stores will close in The company closed Family Dollar stores in the fourth quarter The retailer also reported consolidated net sales for...,4,0.99
Article 3,Having to share personal data turns consumers away from loyalty programs study says,"\nAccording to a new Harris Poll survey, 71% of Americans would be less likely to join a rewards program that collects personal information, such as address, account information and more, with 27%...",According to a new Harris Poll survey of Americans would be less likely to join a rewards program that collects personal information such as address account information and more with saying they w...,3,0.98
Article 4,Walgreens taps Narvar for online pickup returns in 8K stores,\n\np.p1 {margin: 0.0px 0.0px 35.0px 0.0px; line-height: 28.0px; font: 16.0px Helvetica; color: #2a2a2a; -webkit-text-stroke: #2a2a2a; background-color: #ffffff}\r\nspan.s1 {font-kerning: none}\r\...,Customer experience platform Narvar and Walgreens are working together to provide package pickup and returns at more than Walgreens locations that have FedEx OnSite services Its the first offering...,1,0.98
Article 5,TechStyle claims more than 5M active members,"\n\nTechStyle Fashion Group, which operates ShoeDazzle, JustFab, FabKids, Fabletics and Savage X Fenty, now has more than five million active members, according to a TechStyle press release.\n\n\...",TechStyle Fashion Group which operates ShoeDazzle JustFab FabKids Fabletics and Savage X Fenty now has more than five million active members according to a TechStyle press release Program members ...,1,0.85


In [118]:
#Assigning topic names and creating a new column
def topic(x):
    if x['dominant_topic'] == 4:
        return 'Supply Chain/Management changes'
    elif x['dominant_topic'] == 3:
        return 'Loyalty Program/Marketing'
    elif (x['dominant_topic'] == 2) | (x['dominant_topic'] == 1):
        return 'Techonology/Digitilization'
    else:
        return 'Quaterly news/Press release'
    
train_topic_model['topic'] = train_topic_model.apply(topic, axis = 1)
train_topic_model.head()

,title,original_content,content,dominant_topic,contribution %,topic
number,,,,,,
Article 1,Pier 1 appoints interim CFO amid growing financial loss,"\n\nPier 1 on Wednesday reported that fourth quarter net sales fell 19.5% to $412.5 million from $512.2 million in the year-ago quarter. Comp store sales in the quarter decreased 13.7%, which the ...",Pier on Wednesday reported that fourth quarter net sales fell to million from million in the yearago quarter Comp store sales in the quarter decreased which the company attributes partially to the...,4,0.99,Supply Chain/Management changes
Article 2,Family Dollar to close nearly 400 stores,\nDollar Tree on Wednesday announced that up to 390 Family Dollar stores will close in 2019. The company closed 84 Family Dollar stores in the fourth quarter.\nThe retailer also reported consolida...,Dollar Tree on Wednesday announced that up to Family Dollar stores will close in The company closed Family Dollar stores in the fourth quarter The retailer also reported consolidated net sales for...,4,0.99,Supply Chain/Management changes
Article 3,Having to share personal data turns consumers away from loyalty programs study says,"\nAccording to a new Harris Poll survey, 71% of Americans would be less likely to join a rewards program that collects personal information, such as address, account information and more, with 27%...",According to a new Harris Poll survey of Americans would be less likely to join a rewards program that collects personal information such as address account information and more with saying they w...,3,0.98,Loyalty Program/Marketing
Article 4,Walgreens taps Narvar for online pickup returns in 8K stores,\n\np.p1 {margin: 0.0px 0.0px 35.0px 0.0px; line-height: 28.0px; font: 16.0px Helvetica; color: #2a2a2a; -webkit-text-stroke: #2a2a2a; background-color: #ffffff}\r\nspan.s1 {font-kerning: none}\r\...,Customer experience platform Narvar and Walgreens are working together to provide package pickup and returns at more than Walgreens locations that have FedEx OnSite services Its the first offering...,1,0.98,Techonology/Digitilization
Article 5,TechStyle claims more than 5M active members,"\n\nTechStyle Fashion Group, which operates ShoeDazzle, JustFab, FabKids, Fabletics and Savage X Fenty, now has more than five million active members, according to a TechStyle press release.\n\n\...",TechStyle Fashion Group which operates ShoeDazzle JustFab FabKids Fabletics and Savage X Fenty now has more than five million active members according to a TechStyle press release Program members ...,1,0.85,Techonology/Digitilization


In [119]:
#pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(model7, bow_corpus_NN, dictionary_NN)

In [120]:
train_topic_model = train_topic_model[['title', 'original_content', 'content', 'topic']]
train_topic_model.to_csv("Data - Text Classification.csv")